In [25]:
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


from scipy.interpolate import interp1d

from deephyper.analysis._matplotlib import figure_size, update_matplotlib_rc

update_matplotlib_rc()
figsize = figure_size(252 * 1.8, 1.0)

In [15]:
class Experiment:
    task = "navalpropulsion"
    problem = "dhexp.benchmark.hpobench_tabular"
    search = "RANDOM"
    stopper = "deephyper.stopper.ConstantStopper"
    max_evals = 200
    
    stop_step = None
    random_state = None

    def __init__(self, stop_step, random_state) -> None:
        self.stop_step = stop_step
        self.random_state = random_state

    @property
    def log_dir(self):
        return f"{self.task}/output/{self.problem}-{self.search}-{self.stopper}-{self.stop_step}-{self.max_evals}-{self.random_state}"

    @property 
    def path_results(self):
        return os.path.join(self.log_dir, "results.csv")
    
    def load_results(self):
        return pd.read_csv(self.path_results, index_col=None)

In [18]:
random_states = "1608637542 3421126067 4083286876  787846414 3143890026 3348747335 2571218620 2563451924  670094950 1914837113".split()
stop_steps = "1 5 10 15 20 25 30 100".split()

In [23]:
import itertools

df_list = [[] for _ in stop_steps]
for stop_step, random_state in itertools.product(stop_steps, random_states) :
    exp = Experiment(stop_step, random_state)
    df = exp.load_results()
    i = stop_steps.index(stop_step)
    df_list[i].append(df)

In [20]:
from collections import deque

def process(df, mode="max", topk_tournament=None, filter_duplicates=False, max_budget=100):
    assert mode in ["min", "max"]
    
    if df.objective.dtype != np.float64:
        m = df.objective.str.startswith("F")
        df.loc[m,"objective"] = df.loc[m,"objective"].replace("F", "-1000000")
        df = df.astype({"objective": float})
        
    if mode == "min":
        df["objective"] = np.negative(df["objective"])
        if "m:objective_val" in df.columns:
            df["m:objective_val"] = np.negative(df["m:objective_val"])
        df["m:objective_test"] = np.negative(df["m:objective_test"])

    if topk_tournament:
        k = topk_tournament
        max_idx = []
        df["m:budget_cumsum"] = df["m:budget"].cumsum()
        for i in range(len(df)):
            fdf = df[:i+1]
            
            if filter_duplicates:
                fdf = fdf.drop_duplicates([pname for pname in df.columns if "p:" in pname], keep="last")
                
            if mode == "max":
                topk = fdf[:i+1].nlargest(n=k, columns="objective")
                if k == 1:
                    winner = topk
                else:
                    winner = topk.nlargest(n=1, columns="m:objective_val")
            else:
                topk = fdf[:i+1].nsmallest(n=k, columns="objective")
                if k == 1:
                    winner = topk
                else:
                    winner = topk.nsmallest(n=1, columns="m:objective_val")
                
            # consider that checkpointed "weights" can be reloaded
            # df.loc[i,"m:budget_cumsum"] = df.loc[i,"m:budget_cumsum"] + len(topk) * 100 - topk["m:budget"].sum()
            
            # consider that selected models are retrained from scratch
            df.loc[i,"m:budget_cumsum"] = df.loc[i,"m:budget_cumsum"] + len(topk) * max_budget
            
            winner_idx = winner.index.tolist()[0]
            max_idx.append(winner_idx)
        
        df["max_idx"] = max_idx
        
    else:
        if mode == "max":
            df["objective_cummax"] = df["objective"].cummax()
        else:
            df["objective_cummax"] = df["objective"].cummin()
            
        df["m:budget_cumsum"] = df["m:budget"].cumsum()
        df["idx"] = df.index
        df = df.merge(df.groupby("objective_cummax")[["idx"]].first().reset_index(), on="objective_cummax")
        df.rename(columns={"idx_y": "max_idx"}, inplace=True)
        df.index  = df.idx_x.values
        del df["idx_x"]
        
        for idx in df["max_idx"]:
            if df.loc[idx, "m:budget"] < max_budget:
                df.loc[idx, "m:budget_cumsum"] = df.loc[idx, "m:budget_cumsum"] + max_budget
                
    return df

In [ ]:
# https://matplotlib.org/stable/tutorials/colors/colormaps.html#lightness-of-matplotlib-colormaps
cmap = mpl.colormaps["tab10"]

plt.figure()

max_buget = 100
for i in range(len(df_list)):
    color = cmap(i)
    
    x_space = np.arange(1, 200 * max_buget + 1)
    df_list[i] = map(lambda df: process(df, mode="min", topk_tournament=3), df_list[i])
    x_max_i = 0
    y_list = []
    for i, df_i in enumerate(df_list[i]):

        x = df_i["m:budget_cumsum"].to_numpy()
        x_max_i = max(x[-1], x_max_i)

        y = df_i.loc[df_i["max_idx"]]["m:objective_test"].values
        f = interp1d(x, y, kind="previous", fill_value="extrapolate")
        y = f(x_space)

        y_list.append(y)
        
    x = x_space[:x_max_i]
    y_list = np.asarray(y_list)
    y_list = y_list[:, :x_max_i]
    y_mean = y_list.mean(axis=0)
    y_stde = 1.96 * y_list.std(axis=0) / np.sqrt(len(y_list))

    plt.plot(x, y_mean, "-", color=color, label=f"{stop_step[i]}")
    plt.fill_between(x,
                     y_mean - y_stde,
                     y_mean + y_stde,
                     alpha=0.2,
                     color=color)
    
plt.axhline(0.1, linestyle="--", color="lime", linewidth=2)

plt.xlabel("Training Epochs Consumed")
plt.ylabel("Test CE")
# plt.ylabel("Objective")
plt.xscale("log")
# plt.xlim(100, 200 + 3*100)
plt.xlim(200, max_buget*200)
plt.yscale("log")
plt.grid()
plt.grid(visible=True, which="minor", color="gray", linestyle=":")
plt.legend()
plt.savefig(f"figures/pruners_{dataset}_{search}.png", bbox_inches="tight")
plt.show()